In [1]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 

DB Password: ········


In [2]:
#load Cohorts
Case_filtered_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_filtered_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()

In [3]:
#generic function get cohort 
def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

In [8]:
#condition= FIBER Condition 
#df_mrn = pandas Data frame with Patient MRNS 
#name_df = Name o the new Dataframe
#name_feature =Feature name 
#frequency = type of occurence: EVER, COUNT,WINDOW
#Cohort_type = Case/Control
def get_has_certain_condition(condition, df_mrn, name_feature, gap_in_days,window_in_days, frequency, cohort_type):
    #get cohort
    Onset_column = "HT_Onset" if cohort_type == "Case" else "last_encounter"
    cohort = df_to_cohort(df_mrn)
    #get cohort with condition
    cohort_condition = cohort.get(condition)
    # Prefilter condition frame, so there are only the rows of interest --> rows which fulfill the gap condition
    try:
        Filtered_DF = df_mrn.merge(cohort_condition, left_index=True, right_on = "medical_record_number")
    except:
        print("DataFrame is empty. Condition does not fit to any record!")
        raise SystemExit(0)
    Filtered_DF = Filtered_DF.set_index("medical_record_number")
    # Use only necessary columns age_in_days + HT_Onset
    Filtered_DF = Filtered_DF[["age_in_days", Onset_column]]
    Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) >= gap_in_days]
    # Filtered_DF contains all gap relevant entries
    
    # get MRNs of input file 
    #cohort_indexes = df_mrn.index
    #get 0 for no diagnosis and 1 for has diagnosis
    cohort_mrn_diagnosis=[]
    if frequency=='EVER':
        #Remove all duplicate MRNs (index)
        Filtered_DF = Filtered_DF.loc[~Filtered_DF.index.duplicated(keep='first')]
        # Set all Rows to 1, because they occur in the dataframe
        Filtered_DF["has_condition_" + name_feature] = 1
        # Prepare merging to the whole cohort
        to_merge = Filtered_DF["has_condition_" + name_feature]
        to_merge = to_merge.to_frame()
        #Merge it with a left outer join
        cohort_mrn_diagnosis = df_mrn.merge(to_merge, left_index = True, right_index = True, how="left")
        # Fill all Rows, which did not appear in the condition with 0
        cohort_mrn_diagnosis["has_condition_" + name_feature].fillna(0, inplace=True)
    elif frequency=='COUNT':
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    elif frequency=='WINDOW':
        #max window size
        Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) <= (window_in_days+gap_in_days)]
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    else:
        print("function not available")
    #convert list cohort_mrn_diagnosis to panda
    #condition_name= 'has_condition_'+name_feature
    #col_names = ['medical_record_number',condition_name]
    #df_final=pd.DataFrame(cohort_mrn_diagnosis,columns=col_names)
    #df_final.set_index('medical_record_number', inplace=True)
    #df_final=df_mrn.merge( df_final, left_index=True, right_index=True)
    #df_final.to_parquet(name_df)
    #print(df_final)
    df_final = cohort_mrn_diagnosis
    return df_final

In [9]:
condition_hypersomnia = (Diagnosis("327%", "ICD-9")|
                  Diagnosis("G47%", "ICD-10"))

In [10]:
hypersomnia_Window = get_has_certain_condition(condition_hypersomnia, Case_filtered_15_540, "hypersomnia", 180 ,720, "WINDOW" ,"Case")


Fetching data for Diagnosis (...)


In [12]:
hypersomnia_Window.loc[hypersomnia_Window['number_of_occurences_hypersomnia'] >= 1]


,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_hypersomnia
medical_record_number,,,,,,,,,,,,,
1232525234,258,20853.0,21840.0,20853.0,20853.0,Age_BP_condition,17997,2856.0,1954,White,Catholic,Male,6.0
1261168151,60,NaN,17957.0,17957.0,17957.0,Age_ICD_condition,16300,1657.0,1966,White,,Male,1.0
1475298757,172,18633.0,NaN,17809.0,17809.0,Age_MED_condition,15500,2309.0,1965,Other,Catholic,Female,2.0
1519722812,126,27974.0,27779.0,27779.0,27779.0,Age_ICD_condition,26361,1418.0,1939,White,Unknown,Male,1.0
1840280326,566,23903.0,24847.0,24874.0,23903.0,Age_BP_condition,19675,4228.0,1947,White,Presbyterian,Female,1.0
1861213387,40,NaN,16985.0,16990.0,16985.0,Age_ICD_condition,16153,832.0,1969,White,,Female,4.0
1967956140,252,18339.0,NaN,18192.0,18192.0,Age_MED_condition,15504,2688.0,1965,Unknown,Other,Male,8.0
2016151686,432,8749.0,8918.0,8919.0,8749.0,Age_BP_condition,4054,4695.0,1989,Other,None,Female,1.0
2035508486,253,19416.0,NaN,18813.0,18813.0,Age_MED_condition,13814,4999.0,1965,White,Catholic,Male,6.0


In [13]:
hypersomnia_Window.to_parquet('Hypersomniawindow_2yr_case.parquet')


In [14]:
##### CONTROLS######


In [17]:
hypersomnia_Window = get_has_certain_condition(condition_hypersomnia, Control_filtered_15_540, "hypersomnia",  180 ,720, "WINDOW", "Control")


Fetching data for Diagnosis (...)


In [18]:
hypersomnia_Window.loc[hypersomnia_Window['has_condition_hypersomnia'] >= 1]


KeyError: 'has_condition_hypersomnia'

In [ ]:
hypersomnia_Window.to_parquet('Hypersomniawindow_2yr_control.parquet')